In [11]:
import pandas as pd
from IPython.display import display, HTML
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
import matplotlib.pyplot as plt
import numpy as np
import datetime

np.random.seed(42)

In [12]:
from math import pi
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file

output_notebook()

Loading BokehJS ...

In [13]:
SECONDS_IN_MINUTE = 60

In [14]:
# Force CPU usage
import tensorflow as tf
from keras import backend as K

num_cores = 8

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : 1, 'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [15]:
# For plot

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    ax1.set_ylabel('categorical cross entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax2.plot(history.history['binary_accuracy'], label = "training")
    finalize_standardplot(fig, ax1, ax2)
    return fig

In [16]:
def aggregate_market_values(dataframe, aggregation_period):
    """
    timestamp / open / high / low / close / volume btc / volume currency / weighted price
    """
    aggregation_factor = aggregation_period * SECONDS_IN_MINUTE
    
    ret = dataframe.groupby(dataframe.Timestamp // aggregation_factor).agg({
        'Open' : 'first',
        'High' : np.max,
        'Low' : np.min,
        'Close' : 'last',
        'Volume_(BTC)' : np.sum ,
        'Volume_(Currency)' : np.sum,
        'Weighted_Price' : np.mean,
    }).reset_index()
    
    ret.Timestamp *= aggregation_factor
    
    return ret
    

In [17]:
df_raw = pd.read_csv('Data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')

In [18]:
df_raw.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
2,1325318040,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
3,1325318100,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
4,1325318160,4.39,4.39,4.39,4.39,0.455581,2.0,4.39


In [19]:
display(aggregate_market_values(df_raw[:100].copy(), 60))

,Timestamp,Weighted_Price,Volume_(Currency),Open,High,Volume_(BTC),Close,Low
0,1325314800,4.39,16.000000,4.39,4.39,3.644647,4.39,4.39
1,1325318400,4.39,120.000001,4.39,4.39,27.334852,4.39,4.39
2,1325322000,4.39,64.000001,4.39,4.39,14.578588,4.39,4.39


In [ ]:
df = df_raw.drop('Timestamp',1)
df[['Open','High','Low','Close']] = df[['Open','High','Low','Close']].apply(lambda x: np.log(x))
df = (df-df.mean())/df.std()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
def add_MA_n_days_age(num_days):
    num_days_str = str(num_days)
    
    df[['Open_W_MA_'+num_days_str,'High_W_MA_'+num_days_str,'Low_W_MA_'+num_days_str,'Close_W_MA_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).mean()
    df[['Open_MA_'+num_days_str,'High_MA_'+num_days_str,'Low_MA_'+num_days_str,'Close_MA_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).mean()
    df[['Open_EMA_'+num_days_str,'High_EMA_'+num_days_str,'Low_EMA_'+num_days_str,'Close_EMA_'+num_days_str]] = df[['Open','High','Low','Close']].ewm(span=day_window * num_days).mean()

    df[['Open_MAX_'+num_days_str,'High_MAX_'+num_days_str,'Low_MAX_'+num_days_str,'Close_MAX_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).max()
    df[['Open_MIN_'+num_days_str,'High_MIN_'+num_days_str,'Low_MIN_'+num_days_str,'Close_MIN_'+num_days_str]] = df[['Open','High','Low','Close']].rolling(window=day_window * num_days).min()

    df['Open_TENKAN_'+num_days_str] = 0.5 * (df['Open_MIN_'+num_days_str] + df['Open_MAX_'+num_days_str])
    df['High_TENKAN_'+num_days_str] = 0.5 * (df['High_MIN_'+num_days_str] + df['High_MAX_'+num_days_str])
    df['Low_TENKAN_'+num_days_str] = 0.5 * (df['Low_MIN_'+num_days_str] + df['Low_MAX_'+num_days_str])
    df['Close_TENKAN_'+num_days_str] = 0.5 * (df['Close_MIN_'+num_days_str] + df['Close_MAX_'+num_days_str])

def add_prices_n_days_ago(num_days):
    num_days_str = str(num_days)
    df[['Open_S_'+num_days_str,'High_S_'+num_days_str,'Low_S_'+num_days_str,'Close_S_'+num_days_str]] = df[['Open','High','Low','Close']].shift(day_window * num_days)

In [ ]:
day_window = 60 * 24

df[['Open_W','High_W','Low_W','Close_W']] = df[['Open','High','Low','Close']].divide(df['Volume_(BTC)'],axis = 0)

add_prices_n_days_ago(1)
add_prices_n_days_ago(2)
add_prices_n_days_ago(3)

add_MA_n_days_age(5)
add_MA_n_days_age(10)
add_MA_n_days_age(15)

plusieurs y gains selon différents temps + vote

In [ ]:
mean = (df['Open'] + df['Close']) * 0.5
res = mean.shift(day_window)

df['y'] = ((mean - res) > 0).astype(int)

In [ ]:
display(df.sample(10))

In [ ]:
df = df.dropna(how ='any')

y = df.y
df = (df-df.mean())/df.std()
df.y = y
df = df.reset_index()
df = df.drop('index',1)
display(df.describe())

In [22]:
df_plot = aggregate_market_values(df_raw[-1000:].copy(), 60)

df_plot['Date'] = pd.to_datetime(df_plot.Timestamp, unit='s')

display(df_plot.head())

inc = df_plot.Close >= df_plot.Open
dec = df_plot.Open > df_plot.Close
barWidth = 0.66 * 60 * 60*1000 # one minute in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=990, title = "MSFT Candlestick")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df_plot.Date, df_plot.High, df_plot.Date, df_plot.Low, color="black")
p.vbar(df_plot.Date[inc], barWidth, df_plot.Open[inc], df_plot.Close[inc], fill_color="#48D922", line_color="black")
p.vbar(df_plot.Date[dec], barWidth, df_plot.Open[dec], df_plot.Close[dec], fill_color="#FF2828", line_color="black")

show(p)

,Timestamp,Weighted_Price,Volume_(Currency),Open,High,Volume_(BTC),Close,Low,Date
0,1515308400,16621.247639,2.347571e+06,16525.12,16761.26,141.159594,16738.99,16503.02,2018-01-07 07:00:00
1,1515312000,16787.868521,3.977948e+06,16738.99,16855.06,236.913212,16749.30,16730.00,2018-01-07 08:00:00
2,1515315600,16720.176849,3.680132e+06,16749.30,16805.82,220.161403,16640.00,16629.76,2018-01-07 09:00:00
3,1515319200,16744.300496,2.187759e+06,16640.00,16818.05,130.678636,16787.44,16626.83,2018-01-07 10:00:00
4,1515322800,16722.332657,3.778041e+06,16785.10,16802.60,226.116195,16660.13,16644.86,2018-01-07 11:00:00


In [ ]:
length_test2=int(0.8*len(df.index))
window = day_window*30 * 2

index = df.index
index= index[0:length_test2]

indexList = [index[i:min(i + window,len(index))] for i in range(0, len(index), window)]
trainIndex = []
testIndex = []

for i in range(len(indexList)):
    if i%9 != 0:
        trainIndex += indexList[i].tolist()
    else:
        testIndex  += indexList[i].tolist()
        
print(len(trainIndex+testIndex) == len(index))
print(len(indexList))

In [ ]:
train=df.iloc[trainIndex,:]
test1=df.iloc[testIndex,:]
test2=df.drop(train.index).drop(test1.index)

In [ ]:
train_x = train.drop('y',1)
train_y = train['y']
test_x1 = test1.drop('y',1)
test_y1 = test1['y']
test_x2 = test2.drop('y',1)
test_y2 = test2['y']

model = Sequential()
model.add(Dense(128, input_dim=95, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

history = model.fit(train_x, train_y, epochs = 1, verbose=1)
scores1 = model.evaluate(test_x1, test_y1, verbose=0)
scores2 = model.evaluate(test_x2, test_y2, verbose=0)

model.save('my_model2.h5')

In [ ]:
plot_history(history,"test")

In [ ]:
print(scores1)
print(scores2)

In [ ]:
from keras.models import load_model
 
    

with tf.device('/cpu:0'):

    model = load_model('my_model.h5')
    scores = model.evaluate(test_x, test_y, verbose=0)
    print(scores)
    
    del model
